In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [4]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [5]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as dafnetamayo

Initialized MLflow to track repo "dafnetamayo/nyc-taxi-time-prediction"

Repository dafnetamayo/nyc-taxi-time-prediction initialized!

https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/5b5314a67f894d4d9aa944625869aa1c', creation_time=1726630094011, experiment_id='0', last_update_time=1726630094011, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")


In [9]:
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np

## Random forest

In [10]:
mlflow.sklearn.autolog()
 
def objective_rf(params):
    with mlflow.start_run(nested=True):
        # Set model tag
        mlflow.set_tag("model_family", "random_forest")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train RandomForest model
        rf_model = RandomForestRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        rf_model.fit(X_train, y_train)
        
        # Predict on validation dataset
        y_pred = rf_model.predict(X_val)
        
        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        # Log RMSE metric
        mlflow.log_metric("rmse", rmse)
 
    return {'loss': rmse, 'status': STATUS_OK}
 
# Define search space for RandomForest
search_space_rf = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),
    'max_depth': hp.quniform('max_depth', 5, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 8, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 3, 1),
}
 
# Run hyperparameter optimization
with mlflow.start_run(run_name="Random Forest", nested=True):
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best parameters
    mlflow.log_params(best_params_rf)

2024/09/20 15:44:43 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 15:44:46 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 15:45:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID d6c0ce2c1f414c7980e59b932355b1c8. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 15:45:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-stork-952 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/d6c0ce2c1f414c7980e59b932355b1c8.

2024/09/20 15:45:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



 10%|█         | 1/10 [00:18<02:46, 18.48s/trial, best loss: 5.421237858812696]

2024/09/20 15:45:05 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 15:45:12 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID a9c71dc53e2841749ece654deb960d5d. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 15:45:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run adorable-fox-777 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/a9c71dc53e2841749ece654deb960d5d.

2024/09/20 15:45:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



...

2024/09/20 15:46:34 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 15:46:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 9d9e5b00d70c4628a0407520e6d377e5. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 15:46:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run gifted-pig-173 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/9d9e5b00d70c4628a0407520e6d377e5.

2024/09/20 15:46:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



 90%|█████████ | 9/10 [02:03<00:13, 13.81s/trial, best loss: 5.421237858812696]

2024/09/20 15:46:50 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 15:46:59 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 8f2a5f100b3e4876ac82f4258dcfb44d. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 15:46:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run chill-ox-44 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/8f2a5f100b3e4876ac82f4258dcfb44d.

2024/09/20 15:46:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



100%|██████████| 10/10 [02:14<00:00, 13.41s/trial, best loss: 5.421237858812696]


2024/09/20 15:47:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/95850359032a4159812854ee3465e5c9.
2024/09/20 15:47:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.


In [12]:
run_id = "a9c71dc53e2841749ece654deb960d5d"
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/20 15:49:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 2
Created version '2' of model 'nyc-taxi-model'.


## Gradient Boost

In [13]:
mlflow.sklearn.autolog()
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
 
def objective_gb(params):
    with mlflow.start_run(nested=True):
        # Set model tag
        mlflow.set_tag("model_family", "gradient_boosting")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train GradientBoosting model
        gb_model = GradientBoostingRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            learning_rate=float(params['learning_rate']),
            random_state=42
        )
        gb_model.fit(X_train, y_train)
        
        # Predict on validation dataset
        y_pred = gb_model.predict(X_val)
        
        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        # Log RMSE metric
        mlflow.log_metric("rmse", rmse)
 
    return {'loss': rmse, 'status': STATUS_OK}
 
# Define search space for GradientBoosting
search_space_gb = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),
    'max_depth': hp.quniform('max_depth', 3, 8, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 6, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 3, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1)
}
 
# Run hyperparameter optimization for GradientBoosting
with mlflow.start_run(run_name="Gradient Boosting", nested=True):
    best_params_gb = fmin(
        fn=objective_gb,
        space=search_space_gb,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best parameters
    mlflow.log_params(best_params_gb)

2024/09/20 15:59:10 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 15:59:11 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 15:59:22 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 8cfc39b505394ec893b3b0792f0bafde. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 15:59:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run fun-dove-795 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/8cfc39b505394ec893b3b0792f0bafde.

2024/09/20 15:59:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



 10%|█         | 1/10 [00:12<01:54, 12.75s/trial, best loss: 5.375466523736079]

2024/09/20 15:59:23 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 15:59:30 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 0454d93bbbdb4ca999b942904dfa8d3e. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 15:59:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run skittish-duck-865 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/0454d93bbbdb4ca999b942904dfa8d3e.

2024/09/20 15:59:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



...

2024/09/20 16:00:29 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 16:00:35 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 7cd8b6530cc241398278de2c0ba0a0d5. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 16:00:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run charming-snipe-935 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/7cd8b6530cc241398278de2c0ba0a0d5.

2024/09/20 16:00:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



 90%|█████████ | 9/10 [01:25<00:09,  9.04s/trial, best loss: 5.271957658663778]

2024/09/20 16:00:36 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/20 16:00:41 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 200b7076ebba4ecdb08fc140e507f807. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/20 16:00:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run salty-ape-266 at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/200b7076ebba4ecdb08fc140e507f807.

2024/09/20 16:00:42 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.



100%|██████████| 10/10 [01:32<00:00,  9.23s/trial, best loss: 5.271957658663778]


2024/09/20 16:00:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run Gradient Boosting at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/3eb039be095d424da79f770e90362b16.
2024/09/20 16:00:42 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dafnetamayo/nyc-taxi-time-prediction.mlflow/#/experiments/0.


In [14]:

run_id = "0454d93bbbdb4ca999b942904dfa8d3e"
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/20 16:02:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 3
Created version '3' of model 'nyc-taxi-model'.
